In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model as lm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [2]:
df_train_new = pd.read_csv("New_Features.csv")

In [3]:
df_train_new['is_train'] = np.random.uniform(0, 1, len(df_train_new)) <= .75
train, validate = df_train_new[df_train_new['is_train']==True], df_train_new[df_train_new['is_train']==False]

In [4]:
#store gap values
y_train = train.gap.values
y_validate = validate.gap.values
#delete 'gap','is_train', smiles' columns
train = train.drop(['gap'], axis=1)
train = train.drop(['is_train'], axis=1)
train = train.drop(['smiles'], axis=1)
train = train.drop(['Unnamed: 0'], axis=1)
validate = validate.drop(['gap'], axis=1)
validate = validate.drop(['is_train'], axis=1)
validate = validate.drop(['smiles'], axis=1)
validate = validate.drop(['Unnamed: 0'], axis=1)

In [5]:
x_train = train.values
x_validate = validate.values

In [6]:
good_cols = np.nonzero(x_train.sum(axis=0) > 1000)

In [7]:
x_train_small = x_train[:,good_cols[0]]

In [8]:
x_train_small.shape

(749712, 1212)

In [ ]:
rf = RandomForestRegressor()
rf.fit(x_train_small, y_train)

###Is there improvement in RMSE using the training data?

In [4]:
df_train_new.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,2040,2041,2042,2043,2044,2045,2046,2047,gap,smiles
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.19,c1ccc(o1)-c1ccc(s1)-c1cnc(-c2scc3[se]ccc23)c2n...
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.60,C1=CC=C(C1)c1cc2ncc3c4[SiH2]C=Cc4ncc3c2c2=C[Si...
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.49,[nH]1c-2c([SiH2]c3cc(-c4scc5C=CCc45)c4nsnc4c-2...
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.36,[nH]1c2-c3occc3Cc2c2c1cc(-c1cccc3=C[SiH2]C=c13...
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1.98,c1cnc2c3oc4cc(-c5ncncn5)c5nsnc5c4c3c3cocc3c2c1


In [4]:
train_shuff = df_train_new.iloc[np.random.permutation(len(df_train_new))]

In [5]:
train_shuff = train_shuff.drop(['smiles'], axis=1)
sh_train = train_shuff[:500000]
sh_test = train_shuff[500001:]

In [6]:
sh_X_train = sh_train.drop(['gap'], axis=1)
sh_Y_train = sh_train.gap.values
sh_X_test = sh_test.drop(['gap'], axis=1)
sh_Y_test = sh_test.gap.values

In [ ]:
sh_X_train = sh_X_train.drop(['Unnamed: 0'], axis=1)

In [8]:
rf = RandomForestRegressor()
rf.fit(sh_X_train, sh_Y_train)
rf_pred = rf.predict(sh_X_test)

In [9]:
print "Random Forest RMSE with new features"
np.sqrt(mean_squared_error(sh_Y_test, rf_pred))

Random Forest RMSE with new features


0.07647029959019018

Here, we partitioned the training dataset so that we could find out if this method of using newly extracted features and random forest would improve our RMSE. We see that there has been a large improvement in RMSE, going from approximately 0.27 to 0.076. So, it seems promising to submit this to the kaggle by predicting on the actual test dataset.

###Do the same on actual test set

In [ ]:
df_test_new = pd.read_csv("New_Features_test.csv")

In [4]:
X_train = df_train_new.drop(['gap'], axis=1)

In [5]:
X_train = df_train_new.drop(['smiles'], axis=1)

In [6]:
y_train = df_train_new.gap.values

In [7]:
X_test = df_test_new.drop(['smiles'], axis=1)

In [ ]:
x_train = X_train.values

In [ ]:
x_test = X_test.values

In [ ]:
bad_cols = np.nonzero(x_train.sum(axis=0) < 100)

In [ ]:
x_train_small = np.delete(x_train, bad_cols, axis=1)

In [ ]:
rf_test = RandomForestRegressor()
rf_test.fit(X_train, y_train)
rf_pred_test = rf_test.predict(X_test)

In [ ]:
def write_to_file(filename, predictions):
    with open(filename, "w") as f:
        f.write("Id,Prediction\n")
        for i,p in enumerate(predictions):
            f.write(str(i+1) + "," + str(p) + "\n")

In [ ]:
write_to_file("rf_with_newfeatures.csv", rf_pred_test)